In [56]:
import numpy as np

from gymnasium import utils
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium.error import Error
from gymnasium.spaces import Box
from gymnasium.utils.env_checker import check_env

In [57]:
class QRPfRA_v3(MujocoEnv, utils.EzPickle):
    metadata = {
        "render_modes": [
            "human",
            "rgb_array",
            "depth_array",
        ],
    }

    def __init__(self, xml_file="/Users/deniz/PycharmProjects/QRPfRA_Senior_Project/QRPfRA/IK_Models/qrpfra_v3_leg_ik_scene_left.xml", frame_skip=1, **kwargs):
        utils.EzPickle.__init__(self, xml_file, frame_skip, **kwargs)

        MujocoEnv.__init__(
            self,
            xml_file,
            frame_skip=frame_skip,
            observation_space=None,  # needs to be defined after
            default_camera_config={},
            **kwargs,
        )

        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

        obs_size = self.data.qpos.size + self.data.qvel.size

        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(obs_size,), dtype=np.float64
        )

    def step(self, action):
        self.do_simulation(action, self.frame_skip)

        observation = self._get_obs()
        reward = 0
        info = {}

        if self.render_mode == "human":
            self.render()

        return observation, reward, False, False, info

    def _get_obs(self):
        #position = self.data.qpos.flat.copy()
        #velocity = self.data.qvel.flat.copy()
        sensor_data = self.data.sensordata.flat.copy()

        return sensor_data

    def reset_model(self):
        qpos = self.init_qpos
        qvel = self.init_qvel
        self.set_state(qpos, qvel)

        observation = self._get_obs()

        return observation

    def _get_reset_info(self):
        return {"works": True}



In [58]:
env = QRPfRA_v3()
check_env(env)
obs = env.reset()
print(obs)


(array([-0.046 ,  0.1435, -0.1245,  0.    ,  0.    ,  0.    ]), {'works': True})


/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:247: UserWarning: WARN: For Box action spaces, we recommend using a symmetric and normalized space (range=[-1, 1] or [0, 1]). See https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html for more information.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:229: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/deniz/miniconda3/envs/QRPfRA_Senior_Project/lib/python3.11/site-packages/gymnasium/utils/env_checker.py:321: UserWarning: WARN: Not able to test alternative render modes due to the environ

In [59]:
leg_dataset = [] # [obs0, obs1, obs2, act0, act1, act2]

In [60]:
# Save the leg_dataset variable to a text file with comma separation
np.savetxt('leg_dataset.txt', leg_dataset, delimiter=',')


In [61]:
for i in range(-100, 101):
    for j in range(-100, 101):
        for k in range(-100, 101):
            #env.render_mode = "human"
            env_obs = []
            for step_cnt in range(0,3):
                obs, reward, done, _, info = env.step([float(m) for m in [i, j, k]])
                if step_cnt == 2:
                    env_obs = obs
            

            leg_dataset.append([env_obs[0], env_obs[1], env_obs[2], i/100, j/100, k/100])


env.close()

In [62]:
print(len(leg_dataset))

8120601


In [63]:
#Convert left leg list to numpy array
leg_dataset_as_np = np.array(leg_dataset)

In [64]:
print(leg_dataset_as_np[0:4])

observations = leg_dataset_as_np[:, 0:3][:]
print(observations)
actions = leg_dataset_as_np[:, 3:][:]
print(actions)

[[ 0.1291673  -0.17830523 -0.11299864 -1.         -1.         -1.        ]
 [ 0.13793223 -0.15396946 -0.13504426 -1.         -1.         -0.99      ]
 [ 0.13847616 -0.15398756 -0.13407833 -1.         -1.         -0.98      ]
 [ 0.13909475 -0.15248093 -0.13483999 -1.         -1.         -0.97      ]]
[[ 0.1291673  -0.17830523 -0.11299864]
 [ 0.13793223 -0.15396946 -0.13504426]
 [ 0.13847616 -0.15398756 -0.13407833]
 ...
 [ 0.03199582  0.0405381   0.06942399]
 [ 0.03144969  0.03932207  0.06911033]
 [ 0.030937    0.03820823  0.06881587]]
[[-1.   -1.   -1.  ]
 [-1.   -1.   -0.99]
 [-1.   -1.   -0.98]
 ...
 [ 1.    1.    0.98]
 [ 1.    1.    0.99]
 [ 1.    1.    1.  ]]


In [65]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime


# Split the dataset into training and testing sets
train_obs, test_obs, train_actions, test_actions = train_test_split(observations, actions, test_size=0.1, shuffle=True)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='selu'),
    tf.keras.layers.Dense(128, activation='selu'),
    tf.keras.layers.Dense(3, activation='tanh')
])
# Compile the model
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train the model
model.fit(observations, actions, epochs=4, batch_size=128)

# Evaluate the model on the testing set
test_loss = model.evaluate(test_obs, test_actions)
print('Test Loss:', test_loss)
current_time = datetime.datetime.now()
print("Current Time:", current_time)


Epoch 1/4
63443/63443 [==============================] - 49s 769us/step - loss: 0.0211 - accuracy: 0.9100
Epoch 2/4
63443/63443 [==============================] - 53s 837us/step - loss: 0.0174 - accuracy: 0.9230
Epoch 3/4
63443/63443 [==============================] - 50s 791us/step - loss: 0.0165 - accuracy: 0.9281
Epoch 4/4
25377/25377 [==============================] - 10s 396us/step - loss: 0.0174 - accuracy: 0.9211
Test Loss: [0.017432406544685364, 0.9210662841796875]
Current Time: 2024-02-02 02:07:25.875399


In [66]:
model.save("left_legs_model", overwrite=True)

INFO:tensorflow:Assets written to: left_legs_model/assets


INFO:tensorflow:Assets written to: left_legs_model/assets
